<a href="https://colab.research.google.com/github/GabrielYamil/TPs_colab/blob/main/TP2/Ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%%writefile analisisPreferencias.cpp
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <pthread.h>

#define MAX_USERS 100
#define MAX_LINES 1000
#define MAX_FIELD 100
#define ERR_FILE  1
#define OK_FILE   0
#define MAX_LINE  512
#define MAX_GENRE 100
#define NOT_EXIST_IDX -1
#define EXIST_GENRE 2
#define NOT_EXIST_GENRE 3
#define MAX_VIEW 1000
#define MAX_USER_VIEW 200

typedef struct {
    int user_id;
    char user_name[MAX_FIELD];
    char title[MAX_FIELD];
    char type[MAX_FIELD];
    char genre[MAX_FIELD];
} View;

typedef struct {
    int user_id;
    char user_name[MAX_FIELD];
    View* views[MAX_VIEW];
    int view_count;
} UserData;

typedef struct {
    int user_id;
    char user_name[MAX_FIELD];
    char chosen_genre[MAX_FIELD];
    char chosen_type[MAX_FIELD];
    int total;
    int different_genres;
} Preference;

pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
Preference preferences[MAX_USERS];
int idx_preference = 0;



int read_views(const char* filename, View views[], int* total_views);
int logUserViews(View all_views[], int total_views, UserData users[]);
void countType(View* v, int type_count[]);
void countGenre(View* v, int* unique_genres, int genre_counts[], char genres[][MAX_FIELD]);
void type_most_frequent(char chosen_type[], int type_count[]);
int genre_most_frequent(int unique_genres, int genre_count[]);
void saveResult(UserData user, char genres[], char chosen_type[], int unique_genres, int max_index);
int write_json(const char* filename, int total_users);
int find_user_index(int user_id, const UserData users[], int user_count);



int read_views(const char* filename, View views[], int* total_views) {
    FILE* f = fopen(filename, "r");
    if (!f) return ERR_FILE;

    int total_views_c = 0;
    char line[MAX_LINE];

    fgets(line, sizeof(line), f);

    while (fgets(line, sizeof(line), f)) {
        if (total_views_c >= MAX_VIEW) break;

        sscanf(line, "%d,%99[^,],%99[^,],%99[^,],%99[^\n]",
                      &views[total_views_c].user_id,
                      views[total_views_c].user_name,
                      views[total_views_c].title,
                      views[total_views_c].type,
                      views[total_views_c].genre);

        total_views_c++;
    }

    fclose(f);

    *total_views = total_views_c;

    return OK_FILE;
}




int find_user_index(int user_id, const UserData users[], int user_count) {
    for (int i = 0; i < user_count; i++) {
        if (users[i].user_id == user_id)
            return i;
    }
    return NOT_EXIST_IDX;
}




int logUserViews(View all_views[], int total_views, UserData users[])
{
    int user_count = 0;
    for(int i = 0; i < total_views; i++)
    {
      int idx = find_user_index(all_views[i].user_id, users, user_count);
        if (idx == NOT_EXIST_IDX) {
            idx = user_count;
            users[user_count].user_id = all_views[i].user_id;
            strcpy(users[user_count].user_name, all_views[i].user_name);
            users[user_count].view_count = 0;
            user_count++;
        }

        if (users[idx].view_count < MAX_USER_VIEW) {
          users[idx].views[users[idx].view_count++] = &all_views[i];
        }

    }

    return user_count;
}



void countType(View* v, int type_count[])
{
  if (strcmp(v->type, "Serie") == 0)
    type_count[0]++;
  else if (strcmp(v->type, "Película") == 0)
    type_count[1]++;
}




void countGenre(View* v, int* unique_genres, int genre_counts[], char genres[][MAX_FIELD])
{
  int found = NOT_EXIST_GENRE;
  for (int i = 0; i < *unique_genres; i++) {
      if (strcmp(genres[i], v->genre) == 0) {
          genre_counts[i]++;
          found = EXIST_GENRE;
          break;
      }
  }
  if (found == NOT_EXIST_GENRE) {
      strcpy(genres[*unique_genres], v->genre);
      genre_counts[*unique_genres] = 1;
      (*unique_genres)++;
  }
}




void type_most_frequent(char chosen_type[], int type_count[])
{
  strcpy(chosen_type, (type_count[1] >= type_count[0]) ? "Película" : "Serie");
}




int genre_most_frequent(int unique_genres, int genre_counts[])
{
  int max_index = 0;
  for (int i = 1; i < unique_genres; i++) {
      if (genre_counts[i] > genre_counts[max_index]) {
          max_index = i;
      }
  }

  return max_index;
}




void saveResult(UserData user, char genres[][MAX_FIELD], char chosen_type[], int unique_genres, int max_index)
{
    pthread_mutex_lock(&mutex);

    if (idx_preference >= MAX_USERS) {
      pthread_mutex_unlock(&mutex);
      return;
    }

    Preference* pref = &preferences[idx_preference++];

    pref->user_id = user.user_id;
    strcpy(pref->user_name, user.user_name);
    strcpy(pref->chosen_genre, genres[max_index]);
    strcpy(pref->chosen_type, chosen_type);
    pref->total = user.view_count;
    pref->different_genres = unique_genres;
    pthread_mutex_unlock(&mutex);
}




void* analyze_user(void* arg) {
    UserData* user = (UserData*)arg;

    int genre_counts[MAX_GENRE] = {0};
    int type_count[2] = {0}; // 0: Serie, 1: Película
    char genres[MAX_GENRE][MAX_FIELD];
    int unique_genres = 0;

    for (int i = 0; i < user->view_count; i++) {
        View* v = user->views[i];

        countType(v, type_count);
        countGenre(v, &unique_genres, genre_counts, genres);
    }

    char chosen_type[MAX_FIELD];
    type_most_frequent(chosen_type, type_count);
    int max_index = genre_most_frequent(unique_genres, genre_counts);

    UserData userData = *user;
    saveResult(userData, genres, chosen_type, unique_genres, max_index);

    return NULL;
}





int write_json(const char* filename, int total_users) {
    FILE* f = fopen(filename, "w");
    if (!f) return ERR_FILE;

    fprintf(f, "[\n");
    for (int i = 0; i < total_users; i++) {
        Preference* p = &preferences[i];
        fprintf(f,
            "  {\n"
            "    \"user_id\": %d,\n"
            "    \"user_name\": \"%s\",\n"
            "    \"chosen_genre\": \"%s\",\n"
            "    \"chosen_type\": \"%s\",\n"
            "    \"total\": %d,\n"
            "    \"different_genres\": %d\n"
            "  }%s\n",
            p->user_id, p->user_name, p->chosen_genre, p->chosen_type,
            p->total, p->different_genres,
            (i == total_users - 1) ? "" : ","
        );
    }
    fprintf(f, "]\n");
    fclose(f);

    return OK_FILE;
}


int main() {
    View all_views[MAX_VIEW];
    int total_views;
    if (read_views("visualizaciones.csv", all_views, &total_views) == ERR_FILE) {
        perror("No se pudo abrir el archivo CSV");

        return ERR_FILE;
    }

    UserData users[MAX_USERS];
    int user_count = logUserViews(all_views, total_views, users);

    pthread_t threads[MAX_USERS];
    for (int i = 0; i < user_count; i++) {
        if (pthread_create(&threads[i], NULL, analyze_user, &users[i]) != 0) {
          perror("Error creando hilo");
          exit(1);
        }
    }

    for (int i = 0; i < user_count; i++) {
        pthread_join(threads[i], NULL);
    }

    if (write_json("preferencias.json", user_count) == ERR_FILE) {
        perror("Error al escribir JSON");
        return ERR_FILE;
    }

    printf("preferencias.json generado con éxito.\n");
    return 0;
}



Writing analisisPreferencias.cpp


In [2]:
%%writefile visualizaciones.csv
user_id,user_name,title,type,genre
1,p_kiki,Hereditary,Película,Terror
2,daro123,La Casa de Papel,Serie,Drama
1,p_kiki,The Office,Serie,Comedia
1,p_kiki,El Conjuro,Película,Terror

Writing visualizaciones.csv


In [17]:
!rm analisisPreferencias.cpp

In [20]:
!ls -la

total 28
drwxr-xr-x 1 root root 4096 May 12 14:06 .
drwxr-xr-x 1 root root 4096 May 12 13:05 ..
-rw-r--r-- 1 root root 7198 May 12 14:06 analisisPreferencias.cpp
drwxr-xr-x 4 root root 4096 May  8 13:38 .config
drwxr-xr-x 1 root root 4096 May  8 13:38 sample_data
-rw-r--r-- 1 root root  182 May 12 13:17 visualizaciones.csv


In [21]:
!gcc -pthread -o analisis analisisPreferencias.cpp

In [22]:
!./analisis

preferencias.json generado con éxito.


In [24]:
!cat preferencias.json

[
  {
    "user_id": 1,
    "user_name": "p_kiki",
    "chosen_genre": "Terror",
    "chosen_type": "Película",
    "total": 3,
    "different_genres": 2
  },
  {
    "user_id": 2,
    "user_name": "daro123",
    "chosen_genre": "Drama",
    "chosen_type": "Serie",
    "total": 1,
    "different_genres": 1
  }
]
